# 01 — Data Pipeline

Runs pipeline **stages 1–4** to produce training datasets for topic-controlled question generation.

| Stage | Command | Output |
|-------|---------|--------|
| 1 — Convert | `pipe.convert()` | Flat JSON records from SQuAD / KhanQ |
| 2 — Wikify | `pipe.wikify()` | Wikipedia entity annotations |
| 3 — Topics | `pipe.topics()` | Best topic selected per QA pair |
| 4 — Dataset | `pipe.dataset()` | Train/val/test CSVs |

All stages are **idempotent** — re-running a completed stage prints `[SKIP]` and returns immediately.

## Setup

In [ ]:
import sys
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # ── Clone repository ──────────────────────────────────────────────────────
    # TODO: replace with your actual repository URL
    REPO_URL = "https://github.com/YOUR_ORG/YOUR_REPO.git"
    !git clone {REPO_URL} /content/ai4ed-qg -q
    %cd /content/ai4ed-qg

    # ── Install dependencies ──────────────────────────────────────────────────
    !pip install -q transformers sentencepiece requests tqdm pandas \
                    scikit-learn pyyaml python-dotenv

    # ── Mount Google Drive for persistent storage ─────────────────────────────
    # Data survives session restarts when saved to Drive.
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_DIR = Path('/content/drive/MyDrive/ai4ed_qg')
    DRIVE_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Drive mounted. Persistent storage: {DRIVE_DIR}")
else:
    # Running locally — assume cwd is repo root or notebooks/
    DRIVE_DIR = None

import os
project_root = Path('/content/ai4ed-qg') if IN_COLAB else Path.cwd()
if project_root.name == 'notebooks':
    project_root = project_root.parent
os.chdir(project_root)            # pipeline.yaml uses relative paths
sys.path.insert(0, str(project_root))

print(f"Project root : {project_root}")
print(f"Working dir  : {os.getcwd()}")

## API Keys

The Wikify stage calls the Wikifier.org API. Get a free key at https://wikifier.org/register.html.

Store it in Colab Secrets (`🔑` sidebar) under `WIKIFIER_API_KEY`, or set `WIKIFIER_API_KEY` in a local `.env` file.

In [ ]:
if IN_COLAB:
    from google.colab import userdata
    import os
    try:
        os.environ['WIKIFIER_API_KEY'] = userdata.get('WIKIFIER_API_KEY')
        print("Wikifier API key loaded from Colab Secrets")
    except Exception:
        # Fallback: paste key directly
        os.environ['WIKIFIER_API_KEY'] = ""  # ← paste your key here
        print("WARNING: Set WIKIFIER_API_KEY above or add it to Colab Secrets")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
    except ImportError:
        pass

key = os.environ.get('WIKIFIER_API_KEY', '')
if key:
    print(f"WIKIFIER_API_KEY: {key[:6]}... (set)")
else:
    print("WIKIFIER_API_KEY not set — Stage 2 (wikify) will fail")

## Initialise Pipeline

In [ ]:
from src.pipeline import Pipeline

pipe = Pipeline('config/pipeline.yaml')
pipe.status()

## Raw Data

The convert stage expects these files in `data/raw/`:

| File | Source |
|------|--------|
| `train-v1.1.json` | [SQuAD v1.1](https://rajpurkar.github.io/SQuAD-explorer/) |
| `khanq.json` | [KhanQ dataset](https://github.com/topic-qg/khanq) |

For Colab, upload them to Drive and symlink below.

In [ ]:
# ── Colab only: copy raw files from Drive ────────────────────────────────────
import shutil
from pathlib import Path

raw_dir = project_root / 'data' / 'raw'
raw_dir.mkdir(parents=True, exist_ok=True)

if IN_COLAB and DRIVE_DIR:
    for fname in ('train-v1.1.json', 'khanq.json'):
        src = DRIVE_DIR / 'raw' / fname
        dst = raw_dir / fname
        if src.exists() and not dst.exists():
            shutil.copy(src, dst)
            print(f"Copied {fname} from Drive")

for fname in ('train-v1.1.json', 'khanq.json'):
    p = raw_dir / fname
    status = f"{p.stat().st_size / 1e6:.1f} MB" if p.exists() else "MISSING"
    print(f"  {fname}: {status}")

## Stage 1 — Convert

Converts raw SQuAD and KhanQ JSON into flat records ready for wikification.

In [ ]:
pipe.convert(dataset='all')

## Stage 2 — Wikify

Annotates every text passage and question with Wikipedia entity links.

> ⚠️ **SQuAD is large (~90k texts).** Progress is saved every 50 items so it's safe to interrupt and resume. KhanQ (~1,034 texts) finishes in a few minutes.

Choose `tool`:
- `wikifier` — free, just needs `WIKIFIER_API_KEY`
- `wat` — requires WAT token (`WAT_TOKEN` env var)

In [ ]:
WIKIFY_TOOL = 'wikifier'  # 'wikifier' | 'wat'

# Annotate KhanQ first (fast, ~few minutes)
pipe.wikify(dataset='khanq', tool=WIKIFY_TOOL, target='all')

# Annotate SQuAD (slow, run across multiple sessions if needed)
# pipe.wikify(dataset='squad', tool=WIKIFY_TOOL, target='all')

In [ ]:
# ── Colab: sync processed data to Drive so it survives session restart ────────
if IN_COLAB and DRIVE_DIR:
    import shutil
    src = project_root / 'data' / 'processed'
    dst = DRIVE_DIR / 'processed'
    shutil.copytree(src, dst, dirs_exist_ok=True)
    print(f"Synced data/processed/ → {dst}")

## Stage 3 — Topic Selection

For each QA pair, selects the best Wikipedia entity topic using:
1. Intersection of entities annotated in both the text passage and the question
2. Highest relevance score (`pageRank` for Wikifier, `rho` for WAT)

In [ ]:
pipe.topics(dataset='khanq', tool=WIKIFY_TOOL)
# pipe.topics(dataset='squad', tool=WIKIFY_TOOL)

## Stage 4 — Build Datasets

| Mode | Entries | Use |
|------|---------|-----|
| `baseline` | ~full SQuAD | Baseline model (no topic signal) |
| `mixsquad` | 10,000 | TopicQG model training |
| `mixsquad2x` | ~20,000 | TopicQG2X (augmented) training |
| `mixkhanq` | 653 | KhanQ evaluation set |

In [ ]:
# KhanQ evaluation set — needed for the evaluation notebook
pipe.dataset(dataset='khanq', mode='mixkhanq', tool=WIKIFY_TOOL)

# SQuAD training sets — needed for the training notebook
# Uncomment after Stage 2 SQuAD wikification is complete:
# pipe.dataset(dataset='squad', mode='baseline',   tool=WIKIFY_TOOL)
# pipe.dataset(dataset='squad', mode='mixsquad',   tool=WIKIFY_TOOL)
# pipe.dataset(dataset='squad', mode='mixsquad2x', tool=WIKIFY_TOOL)

## Final Status

In [ ]:
pipe.status()

In [ ]:
# ── Colab: final sync of all data to Drive ────────────────────────────────────
if IN_COLAB and DRIVE_DIR:
    import shutil
    for subdir in ('processed', 'training'):
        src = project_root / 'data' / subdir
        if src.exists():
            dst = DRIVE_DIR / subdir
            shutil.copytree(src, dst, dirs_exist_ok=True)
            print(f"Synced data/{subdir}/ → {dst}")

## Next Steps

Proceed to **`02_distillation_training.ipynb`** to fine-tune T5 on the generated datasets.